**Bollinger Bands Trading Strategy Optimization, with specific Hang Sang Index Stocks </br>**

In [24]:
#!pip install yfinance --upgrade
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd ta-lib
#!./configure --prefix=/usr
#!make
#!make install
#!pip install Ta-Lib
#!pip install hkfdb

In [25]:
import os
import sys
import copy
from datetime import datetime, timedelta
import time
import itertools

import pandas as pd
import pandas_datareader as pdr
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import hkfdb
import yfinance as yf
import talib as ta
from talib import MA_Type

## **1) Defining Data Folder Structures & Parameters** 

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#data folder structures
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
backtest_output_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/backtest_output'

**2) EDA with HKFDB**

In [28]:
client = hkfdb.Database(key)

In [29]:
#retrieve all HK stock info, save to google drive
hkstock_db = client.get_basic_hk_stock_info()

# export to CSV file, retaining Chinese language and removing index
#hkstock_db.to_csv('hkstock_db.csv',encoding='utf-8-sig', index=False)

# use stored version of the hkfdb database
#path = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/hkstock_db.csv'
#hkstock_db = pd.read_csv(path, encoding='utf-8-sig')

hkstock_lotsize = hkstock_db[['code', 'lot_size']] 
hkstock_lotsize_dict = copy.deepcopy(hkstock_lotsize.set_index('code')['lot_size'].to_dict())

In [30]:
# Top 10 stocks by turnover
top_30_vol = hkstock_db.sort_values(by='turnover', ascending=False).iloc[0:30]
top_30_vol_lst = top_30_vol['code'].to_list()

In [31]:
# manipulate to ticket list
convert_list = []
for i in top_30_vol_lst:
  new_item = i[1:] + '.HK'
  convert_list.append(new_item)

In [32]:
# pair inverse trade stocks
ticker_list = convert_list

# define date range - yfinance format
start_date = '2011-03-20'
end_date = '2022-12-31'
interval = '1d'

# define date range - hkfdb format
start_date_hkfdb = 20210101
end_date_hkfdb = 20211231
interval_hkfdb = '1D'

# initial capital in HKD
initial_capital = 100000

# position sizing
pos_size = round(initial_capital/len(ticker_list),2)

**2) Getting OHLC Data from Yahoo Finance** </b>


*   Get stock data based on ticker list from Yahoo Finance
*   Create each stock data as different dataframe
*   Add Technical Analysis indicators from TA-Lib
*   Pointing working directory to data folder, export dach dataframe as independent csv file 





In [33]:
hkstock_lotsize = hkstock_db[['code', 'lot_size']] 
hkstock_lotsize_dict = hkstock_lotsize.set_index('code')['lot_size'].to_dict()

In [34]:
# changing working directory to data folder's path
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
os.chdir(data_folder)

hk_ticker_list = []

# getting OHLC data from yfinance package, if auto_adjust=True, OHLC data will not have adj close column, use progress=False to get rid of comments
# Other indicators: 14 day RSI
for ticker in ticker_list:
  try:

    #getting data from y-finance
    hk_ticker = 'HK' + ticker[0:4]
    hk_ticker_list.append(hk_ticker)
    globals()[hk_ticker] = pd.DataFrame
    globals()[hk_ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
    
    # adding technical indicators to the dataframe
    globals()[hk_ticker]['200EMA'] = ta.EMA(globals()[hk_ticker]['Close'], timeperiod=200) 
    globals()[hk_ticker]['ATR'] = ta.ATR(globals()[hk_ticker]['High'], globals()[hk_ticker]['Low'], globals()[hk_ticker]['Close'], timeperiod=14)
    globals()[hk_ticker]['RSI_14'] = ta.RSI(globals()[hk_ticker]['Close'], timeperiod=14)
    globals()[hk_ticker]['MACD'], globals()[hk_ticker]['MACD_signal'], globals()[hk_ticker]['MACD_hist'] = ta.MACD(globals()[hk_ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    globals()[hk_ticker]['ADX'] = ta.ADX(globals()[hk_ticker]['High'], globals()[hk_ticker]['Open'], globals()[hk_ticker]['Close'], 14)
    globals()[hk_ticker]['MFI'] = ta.MFI(globals()[hk_ticker]['High'], globals()[hk_ticker]['Low'], globals()[hk_ticker]['Close'], globals()[hk_ticker]['Volume'], 14)
    globals()[hk_ticker]['WILL%R'] = ta.WILLR(globals()[hk_ticker]['High'], globals()[hk_ticker]['Low'], globals()[hk_ticker]['Close'], timeperiod=14)
    globals()[hk_ticker]['BBand_U'], globals()[hk_ticker]['BBand_M'], globals()[hk_ticker]['BBand_L'] = ta.BBANDS(globals()[hk_ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3

    # add/drop columns and clean up - yfinance/ generic
    globals()[hk_ticker] = globals()[hk_ticker].dropna()
    globals()[hk_ticker] = globals()[hk_ticker].reset_index()
    globals()[hk_ticker][['Position', 'Win Count']] = 0
    globals()[hk_ticker][['Lot Size', 'Equity Value', 'MDD_dollar', 'PNL', 'Holding Period']] = np.NAN
    
    #writing each ticker dataframe into separate .csv files, save to google drive
    #globals()[ticker].to_csv(ticker+'.csv', index=False, header=True)
  except:
    print(globals()[ticker])

**Other Optional Indicators:** </b>

**MACD_hist** - globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9) </b>

**ADX** - globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>


**ADXR** - globals()[ticker]['ADXR'] = ta.ADXR(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>

**50 Day Exponential Moving Average** - globals()[ticker]['50EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=50) </b>

**Bollinger Bands** - globals()[ticker]['BBand_U'], globals()[ticker]['BBand_M'], globals()[ticker]['BBand_L'] = ta.BBANDS(globals()[ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3

**Stochastic Indicators** - globals()[ticker]['Stoch_slowk'], globals()[ticker]['Stoch_slowd'] = ta.STOCH(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'],                                                              fastk_period=21, slowk_period=5, slowk_matype=0, slowd_period=5, slowd_matype=0)    

**3) Building the DataFrame List, Perform Backtesting**

In [35]:
print(hk_ticker_list)

['HK0700', 'HK3690', 'HK9988', 'HK2800', 'HK2828', 'HK2318', 'HK2269', 'HK1299', 'HK1024', 'HK0939', 'HK3968', 'HK0941', 'HK9618', 'HK0388', 'HK1211', 'HK1877', 'HK2628', 'HK1398', 'HK2015', 'HK9999', 'HK3033', 'HK1928', 'HK3988', 'HK0175', 'HK1093', 'HK2331', 'HK0883', 'HK1810', 'HK1177', 'HK6098']


In [36]:
# list of dataframes to be iterated after yfinance data pull 
df_list = [HK0700, HK3690, HK9988, HK2800, HK2828, HK2318, HK2269, HK1299, HK1024, HK0939, HK3968, HK0941, HK9618, HK0388, HK1211, HK1877, HK2628, HK1398, HK2015, HK9999, HK3033, HK1928, HK3988, HK0175, HK1093, HK2331, HK0883, HK1810, HK1177, HK6098]

In [37]:
# function to extract dataframe name
def get_df_name(df):
   name =[x for x in globals() if globals()[x] is df][0]
   return name

In [38]:
# function to print function name as a string
def get_func_name(func_name):
    func_name = sys._getframe().f_code.co_name
    return func_name

**MACD Bollinger Bands Strategy**

*   Entry Rule: Open Price < Lower Bollinger Band, MACD > MACD signal line > 0
*   Exit Rule: Hit profit target/ stop loss/ last day of trade, MACD < signal line



In [39]:
# defining the global list for storing trade list output
global bb_macd_tradelist
bb_macd_tradelist = []

pos_size = round(initial_capital/len(ticker_list),2)

In [40]:
# defining a backtest function based on trading rules, capture all transactions

def backtest_bbmacd(df):
  df_name = get_df_name(df)

  pos_opened = False
  open_price  = 0
  close_price = 0
  lot_size = 0

  initial_equity_value = 0
  win_counter = 0
  open_trade_date = 0
  holding_days = 0

  pnl = 0
  pnl_list = []
    
  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_200EMA      = df.loc[i, '200EMA']
    now_rsi_14      = df.loc[i, 'RSI_14']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_MACD_hist   = df.loc[i, 'MACD_hist']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']
    now_win_count   = df.loc[i, 'Win Count']

    # opening a position
    if (pos_opened == False and (now_open < now_bband_l and now_MACD > now_MACD_signal and now_MACD_hist>0)):
      pos_opened = True
      open_trade_date = now_date
      open_price = now_open
      lot_size = pos_size// open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      df.at[i, 'Position'] = 1
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = initial_equity_value
      bb_macd_tradelist.append([now_date, df_name, pos_opened, open_price, lot_size, initial_equity_value])

    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif (pos_opened == True and ((now_MACD < now_MACD_signal) or \
                                  (lot_size * now_open > initial_equity_value * 1.2) or (lot_size * now_open < initial_equity_value * 0.95) or \
                                  (now_date == df.loc[(len(df)-1), 'Date']))):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      df.at[i, 'Position'] = 2
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = (close_price * lot_size)
      if (now_open * lot_size) < initial_equity_value:
        df.loc[i, 'MDD_dollar']   = -(initial_equity_value - (now_open * lot_size))
      else:
        df.loc[i, 'MDD_dollar']   = 0
      holding_days = now_date - open_trade_date
      df.at[i, 'Holding Period'] = holding_days.days
      
      pnl = round(((close_price * lot_size) - sell_commission) - initial_equity_value,2)
      df.at[i, 'PNL'] = pnl.round(2)
      if pnl > 0:
        df.at[i, 'Win Count'] = 1
        win_counter += 1
      pnl_list.append(pnl)
      bb_macd_tradelist.append([now_date, df_name, pos_opened, close_price, lot_size, (close_price*lot_size), pnl, holding_days])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      lot_size = 0
      open_trade_date = 0

    else:
      df.loc[i, 'Equity Value']   = (now_open * lot_size)
      df.loc[i, 'Lot Size']       = lot_size
      if (now_open * lot_size) < initial_equity_value:
        df.loc[i, 'MDD_dollar']   = initial_equity_value - (now_open * lot_size)
      else:
        df.loc[i, 'MDD_dollar']   = 0

  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].min().round(2)
  avg_hold_period = df['Holding Period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

In [41]:
# create a for loop, put together trade summary list
bbmacd_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_bbmacd(df)
    bbmacd_result_list.append(test_result)

    bbmacd_result_df = pd.DataFrame(bbmacd_result_list)
    bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = bbmacd_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    bbmacd_result_df = pd.DataFrame(bbmacd_result_list)
    bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [106]:
# PNL for Bollinger Bands + MACD Strategy

#measure_date = df_list[1].loc[1, 'Date']
#end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Bollinger Bands + MACD')
#print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((bbmacd_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = bbmacd_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (bbmacd_result_df['No of Trade'].sum()))
print('Total PNL: HKD', bbmacd_result_df['Total PNL'].sum().round(2))
print('Capital Deployed: HKD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(bbmacd_result_df[bbmacd_result_df['No of Trade'] > 0]['Win Rate %'].mean(),2), '%')
bbmacd_result_df[bbmacd_result_df['No of Trade'] > 0].sort_values(by=['Win Rate %', 'Total PNL'], ascending=False)


Trading Strategy: Bollinger Bands + MACD
Total No of Trades Executed: 600
Total PNL: HKD 70269.96
Capital Deployed: HKD 100000
ROI:  70.27 %
Avg Win Rate:  63.47 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
23,HK0175,25,6215.04,248.60,-179.58,10.36,88.00
5,HK2318,23,3614.72,157.16,-145.46,19.61,82.61
21,HK1928,28,3744.10,133.72,-82.88,11.93,82.14
25,HK2331,25,4076.17,163.05,-311.41,10.72,80.00
27,HK1810,8,875.73,109.47,-30.90,9.88,75.00
0,HK0700,30,3807.83,126.93,-124.02,11.90,73.33
11,HK0941,32,2646.59,82.71,-306.69,16.06,71.88
7,HK1299,32,2085.96,65.19,-102.06,12.69,71.88
14,HK1211,30,7923.53,264.12,-221.52,13.27,70.00
13,HK0388,26,3915.89,150.61,-120.41,17.19,69.23


**Scondary Testing with a Refined List, with Win Rate > 65% over 10 years back testing result**

In [114]:
# manipulate ticker name back to stock code
refined_ticker_list = []
second_test_list = bbmacd_result_df[bbmacd_result_df['Win Rate %'] > 65]['Ticker'].to_list()

for i in second_test_list:
  stock_code = i[2:] + '.HK'
  refined_ticker_list.append(stock_code)

In [126]:
# define date range - yfinance format
start_date = '2021-03-10'
end_date = '2022-12-31'
interval = '1d'

# initial capital in HKD
ref_initial_capital = 1000000

# position sizing
ref_pos_size = round(ref_initial_capital/len(ticker_list),2)

In [127]:
# changing working directory to data folder's path
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
os.chdir(data_folder)

refined_hk_ticker_list = []

# getting OHLC data from yfinance package, if auto_adjust=True, OHLC data will not have adj close column, use progress=False to get rid of comments
# Other indicators: 14 day RSI
for ticker in refined_ticker_list:
  try:

    #getting data from y-finance
    hk_ticker = 'HK' + ticker[0:4]
    refined_hk_ticker_list.append(hk_ticker)
    globals()[hk_ticker] = pd.DataFrame
    globals()[hk_ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
    
    # adding technical indicators to the dataframe
    globals()[hk_ticker]['200EMA'] = ta.EMA(globals()[hk_ticker]['Close'], timeperiod=200) 
    globals()[hk_ticker]['ATR'] = ta.ATR(globals()[hk_ticker]['High'], globals()[hk_ticker]['Low'], globals()[hk_ticker]['Close'], timeperiod=14)
    globals()[hk_ticker]['RSI_14'] = ta.RSI(globals()[hk_ticker]['Close'], timeperiod=14)
    globals()[hk_ticker]['MACD'], globals()[hk_ticker]['MACD_signal'], globals()[hk_ticker]['MACD_hist'] = ta.MACD(globals()[hk_ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    globals()[hk_ticker]['ADX'] = ta.ADX(globals()[hk_ticker]['High'], globals()[hk_ticker]['Open'], globals()[hk_ticker]['Close'], 14)
    globals()[hk_ticker]['MFI'] = ta.MFI(globals()[hk_ticker]['High'], globals()[hk_ticker]['Low'], globals()[hk_ticker]['Close'], globals()[hk_ticker]['Volume'], 14)
    globals()[hk_ticker]['WILL%R'] = ta.WILLR(globals()[hk_ticker]['High'], globals()[hk_ticker]['Low'], globals()[hk_ticker]['Close'], timeperiod=14)
    globals()[hk_ticker]['BBand_U'], globals()[hk_ticker]['BBand_M'], globals()[hk_ticker]['BBand_L'] = ta.BBANDS(globals()[hk_ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3

    # add/drop columns and clean up - yfinance/ generic
    globals()[hk_ticker] = globals()[hk_ticker].dropna()
    globals()[hk_ticker] = globals()[hk_ticker].reset_index()
    globals()[hk_ticker][['Position', 'Win Count']] = 0
    globals()[hk_ticker][['Lot Size', 'Equity Value', 'MDD_dollar', 'PNL', 'Holding Period']] = np.NAN
    
    #writing each ticker dataframe into separate .csv files, save to google drive
    #globals()[ticker].to_csv(ticker+'.csv', index=False, header=True)
  except:
    print(globals()[ticker])

In [128]:
hkstock_lotsize['Ticker'] =  'HK' + hkstock_db['code'].str[1:]
#hkstock_lotsize = hkstock_lotsize.drop(columns='index', axis=1)
hkstock_lotsize_dict = hkstock_lotsize.set_index('Ticker')['lot_size'].to_dict()

In [129]:
print(refined_hk_ticker_list)

['HK0700', 'HK2800', 'HK2318', 'HK1299', 'HK0941', 'HK0388', 'HK1211', 'HK1877', 'HK1928', 'HK0175', 'HK2331', 'HK0883', 'HK1810']


In [130]:
refined_df_list = [HK0700, HK2800, HK2318, HK1299, HK0941, HK0388, HK1211, HK1877, HK1928, HK0175, HK2331, HK0883, HK1810]

In [131]:
def get_min_share(df_name):
  min_shares = hkstock_lotsize_dict.get(df_name)
  return min_shares

In [132]:
# defining a backtest function based on trading rules, capture all transactions

def backtest_bbmacd_with_lot(df):
  df_name = get_df_name(df)

  pos_opened = False
  open_price  = 0
  close_price = 0
  lot_size = 0
  one_lot_share = get_min_share(df_name)

  initial_equity_value = 0
  win_counter = 0
  open_trade_date = 0
  holding_days = 0

  pnl = 0
  pnl_list = []
    
  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_200EMA      = df.loc[i, '200EMA']
    now_rsi_14      = df.loc[i, 'RSI_14']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_MACD_hist   = df.loc[i, 'MACD_hist']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']
    now_win_count   = df.loc[i, 'Win Count']

    # opening a position
    if (pos_opened == False and (now_open < now_bband_l and now_MACD > now_MACD_signal and now_MACD_hist>0)):
      pos_opened = True
      open_trade_date = now_date
      open_price = now_open
      lot_size = ref_pos_size// (open_price * one_lot_share)
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price * one_lot_share) - buy_commission),2)
      df.at[i, 'Position'] = 1
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = initial_equity_value
      bb_macd_tradelist.append([now_date, df_name, pos_opened, open_price, lot_size, initial_equity_value])

    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif (pos_opened == True and ((now_MACD < now_MACD_signal) or \
                                  (lot_size * now_open > initial_equity_value * 1.2) or (lot_size * now_open < initial_equity_value * 0.95) or \
                                  (now_date == df.loc[(len(df)-1), 'Date']))):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      df.at[i, 'Position'] = 2
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = (close_price * lot_size * one_lot_share)
      if (now_open * lot_size) < initial_equity_value:
        df.loc[i, 'MDD_dollar']   = -(initial_equity_value - (now_open * lot_size * one_lot_share))
      else:
        df.loc[i, 'MDD_dollar']   = 0
      holding_days = now_date - open_trade_date
      df.at[i, 'Holding Period'] = holding_days.days
      
      pnl = round(((close_price * lot_size * one_lot_share) - sell_commission) - initial_equity_value,2)
      df.at[i, 'PNL'] = pnl.round(2)
      if pnl > 0:
        df.at[i, 'Win Count'] = 1
        win_counter += 1
      pnl_list.append(pnl)
      bb_macd_tradelist.append([now_date, df_name, pos_opened, close_price, lot_size, (close_price*lot_size), pnl, holding_days])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      lot_size = 0
      open_trade_date = 0

    else:
      df.loc[i, 'Equity Value']   = (now_open * lot_size)
      df.loc[i, 'Lot Size']       = lot_size
      if (now_open * lot_size) < initial_equity_value:
        df.loc[i, 'MDD_dollar']   = initial_equity_value - (now_open * lot_size)
      else:
        df.loc[i, 'MDD_dollar']   = 0

  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].min().round(2)
  avg_hold_period = df['Holding Period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

In [133]:
# create a for loop, put together trade summary list, with lot size applied
refined_bbmacd_result_list = []

for df in refined_df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_bbmacd_with_lot(df)
    refined_bbmacd_result_list.append(test_result)

    refined_bbmacd_result_df = pd.DataFrame(refined_bbmacd_result_list)
    refined_bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    refined_test_result_null = refined_bbmacd_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    refined_bbmacd_result_df = pd.DataFrame(bbmacd_result_list)
    refined_bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [137]:
# PNL for Bollinger Bands + MACD Strategy

measure_date = refined_df_list[1].loc[1, 'Date']
end_date = refined_df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Bollinger Bands + MACD')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((refined_bbmacd_result_df['Total PNL'].sum()/ ref_initial_capital)*100).round(2)
MDD = refined_bbmacd_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (refined_bbmacd_result_df['No of Trade'].sum()))
print('Total PNL: HKD', refined_bbmacd_result_df['Total PNL'].sum().round(2))
print('Capital Deployed: HKD', ref_initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(refined_bbmacd_result_df[refined_bbmacd_result_df['No of Trade'] > 0]['Win Rate %'].mean(),2), '%')
refined_bbmacd_result_df[refined_bbmacd_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Win Rate %'], ascending=False)


Trading Strategy: Bollinger Bands + MACD
Trade period: 2021-12-30 to 2022-12-30
Total No of Trades Executed: 43
Total PNL: HKD 24496.98
Capital Deployed: HKD 1000000
ROI:  2.45 %
Avg Win Rate:  81.04 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
10,HK2331,7,6367.85,909.69,0.00,2.71,71.43
0,HK0700,4,3696.32,924.08,0.00,2.75,75.00
8,HK1928,3,3151.61,1050.54,0.00,1.00,100.00
7,HK1877,2,3079.76,1539.88,0.00,2.50,100.00
12,HK1810,4,1910.47,477.62,-415.87,1.50,75.00
9,HK0175,2,1885.79,942.90,0.00,2.00,100.00
3,HK1299,7,1189.45,169.92,-178.97,1.00,57.14
1,HK2800,3,944.73,314.91,0.00,1.00,100.00
5,HK0388,2,854.77,427.38,0.00,1.00,100.00
2,HK2318,2,659.00,329.50,0.00,1.00,100.00
